In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime,gc,math
import random
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
%matplotlib inline

In [ ]:
role_info = pd.read_csv('./data/mr_role_7d.csv')
user_info = pd.read_csv('./user_info.csv')

In [ ]:
def mr_7d_FE():
    def active_time_transform(df):
        df['role_created_active_time'].fillna('"0-8,0","8-12,0","12-14,0","14-18,0","18-24,0"', inplace=True)
        df['role_created_active_time'] = df['role_created_active_time'].apply(lambda x:str(x)[1:-1])
        temp = df['role_created_active_time'].str.split(',',expand=True).iloc[:,[1,3,5,7,9]].rename(columns={1:'active_0-8',
                                            3:'active_8-12',5:'active_12-14',7:'active_14-18',9:'active_18-24'})
        for i in range(len(temp.columns)):
            temp.iloc[:,i] = temp.iloc[:,i].apply(lambda x:str(x)[:-1])
        df = df.join(temp).drop(['role_created_active_time'], axis=1)
        return df

    def pay_grade_transform(df):
        df['pay_grade'].fillna('[0,0,0,0,0,0,0]',inplace=True)
        df['pay_grade'] = df['pay_grade'].apply(lambda x:str(x)[1:-1])
        temp = df['pay_grade'].str.split(',',expand=True).rename(columns={0:'pay_grade_1',
                    1:'pay_grade_2',2:'pay_grade_3',3:'pay_grade_4',4:'pay_grade_5',5:'pay_grade_6',6:'pay_grade_7'})
        df = df.join(temp).drop(['pay_grade'], axis=1)
        return df
    
    role_info = active_time_transform(role_info)
    role_info = pay_grade_transform(role_info)
    role_info['pay_num'].fillna(0, inplace=True)
    role_info['pay_sum'].fillna(0, inplace=True)
    role_info['role_created_active'] = role_info['role_created_active'].clip(0,7)
    role_info['pay_rate'] = role_info['pay_num'] / (role_info['role_created_active'] + 1e-4)
    role_info['pay_avg'] = role_info['pay_sum'] / (role_info['pay_num'] + 1e-4)
    
    # 清洗设备金额
    role_info.loc[role_info['p_model'] == 'iPhone 6s Plus','model_money'] = 3500
    role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 5000
    role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 2500
    role_info.loc[role_info['p_model'] == 'iPhone12,1','model_money'] = 2000
    role_info['model_money'] = role_info['model_money'].replace(0, np.nan)
    role_info['model_money'].fillna(role_info[role_info['model_money'] !=0]['model_money'].mean(),inplace=True)
    bins=[0,1000,1500,2000,2500,3000,3500,4000,4500,5000,6000,8000,10000]
    role_info['model_money_level'] = pd.cut(role_info['model_money'],bins,labels=False)

    # 清洗login_num
    role_info['role_created_login_num'].clip(1,220,inplace=True)
    tmp = role_info[role_info['role_created_login_num'] <3]
    tmp['role_created_login_num'] = tmp['max_role_level'] * 0.6
    role_info['role_created_login_num'].update(tmp['role_created_login_num'])
    role_info['role_created_login_num'].clip(1,220,inplace=True)
    
    # 清洗max_role_level
    tmp = role_info[role_info['max_role_level'] <5]
    tmp['max_role_level'] = tmp['role_created_online'] * 0.001538
    role_info['max_role_level'].update(tmp['max_role_level'])
    role_info['max_role_level'].clip(1,100,inplace=True)
    
    # 清洗online
    tmp = role_info[role_info['role_created_online'] <1800]
    tmp['role_created_online'] = tmp['max_role_level'] * 649.93
    role_info['role_created_online'].update(tmp['role_created_online'])
    role_info['role_created_online'].clip(300,201600,inplace=True)
    
    # 处理时间特征create_role_time
    role_info = role_info.sort_values('create_role_time').reset_index(drop=True)
    role_info['hour'] = role_info['create_role_time'].dt.hour
    role_info['weekend'] = role_info['create_role_time'].dt.weekday
    holidays = ['2019-09-13','2019-09-14','2019-09-15','2019-10-01','2019-10-02','2019-10-03','2019-10-04','2019-10-05','2019-10-06','2019-10-07',
                '2020-01-01','2020-01-24','2020-01-25','2020-01-26','2020-01-27','2020-01-28','2020-01-29','2020-01-30','2020-04-04','2020-04-05',
                '2020-04-06','2020-04-04','2020-05-01','2020-05-02','2020-05-03','2020-05-04','2020-05-05','2020-06-25','2020-06-27']
    role_info['timestamp'] = role_info['create_role_time'].dt.date.apply(lambda x:str(x))
    role_info['is_holidays'] = (role_info['timestamp'].isin(holidays)).astype(int)
    role_info.drop(['timestamp'], axis=1, inplace=True)
    
    # 合并role_user
    data = pd.merge(role_info,user_info,on='user_id',how='left',validate='many_to_one')
    # 标记用户时否绑定了手机，绑定;1    未绑定：0
    data['mobile'] = pd.to_numeric(data['mobile'],errors='coerce')
    data['mobile'] = data['mobile'].apply(lambda x:0 if np.isnan(x) == True else 1)
    # 用户注册时间与角色注册时间的时间差,并进行分桶
    data['reg_date'] = pd.to_datetime(data['reg_date'])
    data['time_interval'] = (data['create_role_time'] - data['reg_date']).dt.total_seconds()/3600
    data['time_interval'] = data['time_interval'].clip(0,2000)
    data['time_interval'] = pd.cut(data['time_interval'], bins=[0,1,3,12,24,72,168,720,2000],labels=False)
    data['active_0-8'] = pd.to_numeric(data['active_0-8'], errors='coerce')
    data['active_8-12'] = pd.to_numeric(data['active_8-12'], errors='coerce')
    data['active_12-14'] = pd.to_numeric(data['active_12-14'], errors='coerce')
    data['active_14-18'] = pd.to_numeric(data['active_14-18'], errors='coerce')
    data['active_18-24'] = pd.to_numeric(data['active_18-24'], errors='coerce')

